# Homework 2

#### Medio ambiente y reproducibilidad

In [63]:
# Librerías necesarias
!pip install streamlit-folium geopandas shapely folium pandas numpy matplotlib chardet

import streamlit as st
from streamlit_folium import st_folium
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
import folium
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster
from pathlib import Path
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import chardet

In [64]:
base = open(r'../data/Datos_panel_distritos.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']
charenc

'utf-8'

In [65]:
# Lectura de archivos
# Detectar codificación del CSV de hospitales
with open("../data/hospitales.csv", "rb") as f:
    base = f.read()
det = chardet.detect(base)
charenc = det["encoding"]
print("Codificación hospitales:", charenc)

# Cragar hospitales y shapefile de distritos
distritos = gpd.read_file("../data/shape_file/DISTRITOS.shp").to_crs("EPSG:4326")
hospitales = pd.read_csv("../data/hospitales.csv", encoding="latin-1")
hospitales = hospitales.dropna(subset=["ESTE", "NORTE"])  # limpiar NaN

print("Ejemplo hospitales:\n", hospitales.head())

Codificación hospitales: MacRoman
Ejemplo hospitales:
           Institución  Código Único         Nombre del establecimiento  \
1   GOBIERNO REGIONAL          7050                             AMBATO   
2   GOBIERNO REGIONAL            99          SANTA ISABEL DE YUMBATURO   
6               MINSA          7278  PUESTO DE SALUD HEROES DEL CENEPA   
12  GOBIERNO REGIONAL          5460                      NUEVA BETANIA   
15  GOBIERNO REGIONAL          6431                         PONGO ISLA   

                         Clasificación  \
1   PUESTOS DE SALUD O POSTAS DE SALUD   
2   PUESTOS DE SALUD O POSTAS DE SALUD   
6   PUESTOS DE SALUD O POSTAS DE SALUD   
12  PUESTOS DE SALUD O POSTAS DE SALUD   
15  PUESTOS DE SALUD O POSTAS DE SALUD   

                                          Tipo Departamento         Provincia  \
1   ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO    CAJAMARCA           CUTERVO   
2   ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO       LORETO            LORETO   
6   

## Análisis geoespacial con geopandas

### TAREA 1: Mapas estáticos: recuento de hospitales por distrito

In [66]:
# 1. Filtración de datos: hospitales públicos operativos

hospitales_publicos = hospitales[
    (hospitales["Estado"].str.upper() == "ACTIVADO") &
    (hospitales["Condición"].str.upper() == "EN FUNCIONAMIENTO") &
    (hospitales["Institución"].str.contains("GOBIERNO|MINSA", case=False, na=False))
].copy()

# Crear GeoDataFrame
hospitales_publicos["geometry"] = hospitales_publicos.apply(
    lambda row: Point(float(row["ESTE"]), float(row["NORTE"]))
    if pd.notna(row["ESTE"]) and pd.notna(row["NORTE"]) else None,
    axis=1
)

hospitales_gdf = gpd.GeoDataFrame(
    hospitales_publicos,
    geometry="geometry",
    crs="EPSG:4326"
)

In [ ]:
print(distritos.head())

  IDDPTO DEPARTAMEN IDPROV    PROVINCIA  IDDIST                DISTRITO  \
0     10    HUANUCO   1009  PUERTO INCA  100902         CODO DEL POZUZO   
1     10    HUANUCO   1009  PUERTO INCA  100904             TOURNAVISTA   
2     25    UCAYALI   2503   PADRE ABAD  250305  ALEXANDER VON HUMBOLDT   
3     25    UCAYALI   2503   PADRE ABAD  250302                 IRAZOLA   
4     25    UCAYALI   2503   PADRE ABAD  250304                 NESHUYA   

                  CAPITAL CODCCPP  AREA FUENTE  \
0         CODO DEL POZUZO    0001     1   INEI   
1             TOURNAVISTA    0001     1   INEI   
2  ALEXANDER VON HUMBOLDT    0001     1   INEI   
3           SAN ALEJANDRO    0001     1   INEI   
4            MONTE ALEGRE    0001     1   INEI   

                                            geometry  
0  POLYGON ((-75.31797 -9.29529, -75.3171 -9.2975...  
1  POLYGON ((-74.64136 -8.82302, -74.64036 -8.828...  
2  POLYGON ((-75.02253 -8.74193, -75.02267 -8.742...  
3  POLYGON ((-75.13864 -8.56

In [69]:
# 2. Conteo de hospitales por distrito

hospitales_count = hospitales_gdf.groupby("UBIGEO").size().reset_index(name="hospitales")

# Asegurar que UBIGEO tenga 6 dígitos
distritos["UBIGEO"] = distritos["IDDIST"].astype(str).str.zfill(6)
hospitales_count["UBIGEO"] = hospitales_count["UBIGEO"].astype(str).str.zfill(6)

# Merge distritos con conteo de hospitales
distritos_hosp = distritos.merge(
    hospitales_count,
    on="UBIGEO",
    how="left"
).fillna({"hospitales": 0})

In [70]:
# 3. Mapas solicitados

# Mapa 1: Total de hospitales públicos por distrito
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
distritos_hosp.plot(
    column="hospitales",
    cmap="YlOrRd",  # escala más clara y progresiva
    legend=True,
    linewidth=0.2,
    edgecolor="black",
    ax=ax,
    legend_kwds={"label": "Número de hospitales", "orientation": "vertical"}
)
plt.title("Mapa 1: Total hospitales públicos por distrito", fontsize=14)
plt.axis("off")
plt.savefig("../outputs/mapa1.png", dpi=300, bbox_inches="tight")
plt.show()

# Mapa 2: Distritos con cero hospitales
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
distritos_hosp.plot(color="whitesmoke", edgecolor="black", linewidth=0.2, ax=ax)
distritos_hosp[distritos_hosp["hospitales"] == 0].plot(
    color="red", ax=ax, label="Distritos sin hospitales"
)
plt.title("Mapa 2: Distritos con cero hospitales", fontsize=14)
plt.legend()
plt.axis("off")
plt.savefig("../outputs/mapa2.png", dpi=300, bbox_inches="tight")
plt.show()

# Mapa 3: Top 10 distritos con más hospitales
top10 = distritos_hosp.nlargest(10, "hospitales")

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
distritos_hosp.plot(color="lightgrey", edgecolor="black", linewidth=0.2, ax=ax)
top10.plot(
    column="hospitales",
    cmap="Reds",
    legend=True,
    edgecolor="black",
    linewidth=0.5,
    ax=ax,
    legend_kwds={"label": "Hospitales (Top 10)", "orientation": "vertical"}
)
plt.title("Mapa 3: Top 10 distritos con más hospitales", fontsize=14)
plt.axis("off")
plt.savefig("../outputs/mapa3.png", dpi=300, bbox_inches="tight")
plt.show()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13256\2844439225.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13256\2844439225.py:26: UserWarning: Legend does not support handles for PatchCollection instances.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#implementing-a-custom-legend-handler
  plt.legend()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13256\2844439225.py:26: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13256\2844439225.py:29: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13256\2844439225.py:48: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### TAREA 2: Análisis a nivel departamental

In [71]:
# 1. Conteo de hospitales por departamento

dpto_hosp = hospitales.groupby("Departamento").size().reset_index(name="hospitales")

# Ordenar de mayor a menor
dpto_hosp = dpto_hosp.sort_values("hospitales", ascending=False).reset_index(drop=True)

print("Resumen de hospitales por departamento:")
print(dpto_hosp)

# Guardar tabla resumen en CSV
dpto_hosp.to_csv("../outputs/resumen_departamentos.csv", index=False, encoding="utf-8")

# Identificación de máximo y mínimo
max_dept = dpto_hosp.iloc[0]
min_dept = dpto_hosp.iloc[-1]

print(f"Depto con MÁS hospitales: {max_dept['Departamento']} ({max_dept['hospitales']})")
print(f"Depto con MENOS hospitales: {min_dept['Departamento']} ({min_dept['hospitales']})")

Resumen de hospitales por departamento:
     Departamento  hospitales
0       CAJAMARCA         849
1            LIMA         776
2           JUNIN         493
3           PIURA         444
4          ANCASH         418
5        AMAZONAS         417
6            PUNO         407
7        AYACUCHO         387
8      SAN MARTIN         379
9        APURIMAC         375
10         LORETO         353
11          CUSCO         339
12    LA LIBERTAD         336
13   HUANCAVELICA         325
14        HUANUCO         261
15          PASCO         258
16        UCAYALI         234
17       AREQUIPA         224
18     LAMBAYEQUE         183
19            ICA         146
20         CALLAO          97
21          TACNA          79
22  MADRE DE DIOS          66
23       MOQUEGUA          56
24         TUMBES          54
Depto con MÁS hospitales: CAJAMARCA (849)
Depto con MENOS hospitales: TUMBES (54)


In [72]:
# 2. Gráfico de barras

plt.figure(figsize=(14, 7))
bars = plt.bar(dpto_hosp["Departamento"], dpto_hosp["hospitales"], color="skyblue", edgecolor="black")
plt.xticks(rotation=90)
plt.title("Número de hospitales por departamento", fontsize=14, fontweight="bold")
plt.xlabel("Departamento")
plt.ylabel("Cantidad de hospitales")

# Añadir etiquetas encima de cada barra
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 2, int(yval),
             ha="center", va="bottom", fontsize=8)

plt.tight_layout()
plt.savefig("../outputs/mapa4.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13256\1056992955.py:18: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [76]:
# 3. Mapa coroplético por departamento

# Agrupar el shapefile a nivel de departamento
departamentos = distritos.dissolve(by="DEPARTAMEN").reset_index()

# Merge con conteo de hospitales
departamentos = departamentos.merge(dpto_hosp, left_on="DEPARTAMEN", right_on="Departamento", how="left")
departamentos["hospitales"] = departamentos["hospitales"].fillna(0)

# Plot del mapa
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
departamentos.plot(
    column="hospitales",
    cmap="YlGnBu",
    legend=True,
    edgecolor="black",
    linewidth=0.5,
    ax=ax,
    legend_kwds={"label": "Número de hospitales", "orientation": "vertical"}
)

plt.title("Mapa de hospitales por departamento", fontsize=14, fontweight="bold")
plt.axis("off")
plt.savefig("../outputs/mapa5.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13256\4265138260.py:25: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### TAREA 3: Análisis de proximidad (utilizando centros de población)

In [ ]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import chardet
import folium
from folium.plugins import MarkerCluster
import os
import warnings

warnings.filterwarnings("ignore")

# 0) Localizar la raíz del proyecto
def find_project_root(max_up=6):
    p = Path.cwd()
    for _ in range(max_up):
        if (p / "data").exists():
            return p
        p = p.parent
    return Path.cwd()

PROJECT_ROOT = find_project_root()
print("Project root detectado en:", PROJECT_ROOT)

# 1) Paths y configuración
BASE_DIR = PROJECT_ROOT
DATA_DIR = BASE_DIR / "data"
SHAPE_DIR = DATA_DIR / "shape_file"
OUT_DIR = BASE_DIR / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

EXPECTED_POP = "CCPP_IGN100K.shp"
POP_FILE = SHAPE_DIR / EXPECTED_POP
DISTRICTS_FILE = SHAPE_DIR / "DISTRITOS.shp"
HOSP_CSV = DATA_DIR / "hospitales.csv"

print("DATA_DIR:", DATA_DIR)
print("SHAPE_DIR:", SHAPE_DIR)
print("OUT_DIR:", OUT_DIR)
print("Archivo hospitales esperado:", HOSP_CSV, "exists:", HOSP_CSV.exists())
print("Archivo centros poblados esperado:", POP_FILE, "exists:", POP_FILE.exists())

if not POP_FILE.exists():
    shp_files = [f for f in SHAPE_DIR.glob("*.shp")]
    print("Shapefiles disponibles en shape_file:", [p.name for p in shp_files])
    candidate = None
    for p in shp_files:
        name = p.name.upper()
        if any(x in name for x in ["CCPP", "CCDD", "CENTRO", "CENTROS", "CCPP_IGN", "CCPP_IGNI"]):
            candidate = p
            break
    if candidate is None and len(shp_files) > 0:
        for p in shp_files:
            if "CC" in p.name.upper():
                candidate = p
                break
    if candidate is None and len(shp_files) > 0:
        candidate = shp_files[0]
    if candidate:
        print("Usando shapefile detectado para centros poblados:", candidate.name)
        POP_FILE = candidate
    else:
        raise FileNotFoundError(f"No se encontró ningún .shp en {SHAPE_DIR}. Añade el shapefile de centros poblados.")

# CRS
CRS_WGS84 = "EPSG:4326"
CRS_UTM18S = "EPSG:32718"

# 2) Funciones
def cargar_hospitales_csv(csv_path):
    """Carga CSV de hospitales y devuelve GeoDataFrame en CRS WGS84 (filtra coordenadas y estados)."""
    if not csv_path.exists():
        raise FileNotFoundError(f"No se encontró el CSV de hospitales en {csv_path}")
    with open(csv_path, "rb") as f:
        raw = f.read()
    enc = chardet.detect(raw)["encoding"] or "latin-1"
    df = pd.read_csv(csv_path, encoding=enc, dtype=str)
    print(f"[Hosp] Registros originales: {len(df)}")

    # Normaliza columnas
    df.columns = [c.strip() for c in df.columns]

    # Detecta columnas de lon/lat
    lon_col = next((c for c in df.columns if c.upper() in ["ESTE", "LONG", "LON", "X"]), None)
    lat_col = next((c for c in df.columns if c.upper() in ["NORTE", "LAT", "Y"]), None)
    if lon_col is None or lat_col is None:
        raise ValueError("No se pudieron encontrar columnas de coordenadas (ESTE/NORTE) en hospitales.")

    # Convertir a numérico y filtrar nulos
    df[lon_col] = pd.to_numeric(df[lon_col], errors="coerce")
    df[lat_col] = pd.to_numeric(df[lat_col], errors="coerce")
    df = df.dropna(subset=[lon_col, lat_col])
    print(f"[Hosp] Con coordenadas válidas: {len(df)}")

    # Rango plausible para Perú
    df = df[(df[lat_col].between(-20, 5)) & (df[lon_col].between(-85, -68))]
    print(f"[Hosp] En rango Perú: {len(df)}")

    # Normalizar texto en columnas relevantes
    for col in ["Estado", "Condición", "Institución", "SITUACION", "SITUACIÓN"]:
        if col in df.columns:
            df[col] = df[col].str.strip().str.upper()

    # Mostrar valores disponibles en 'Estado'/'Condición' para que ajustes si hace falta
    check_cols = {}
    for col in ["Estado", "Condición", "Institución"]:
        if col in df.columns:
            check_cols[col] = df[col].dropna().unique().tolist()
    print("[Hosp] Valores detectados (Estado/Condición/Institución):", check_cols)

    # Aplicar máscara: intentar filtrar operativos/activos del MINSA/Gobierno
    mask = pd.Series(True, index=df.index)
    if "Condición" in df.columns:
        mask &= df["Condición"].isin(["EN FUNCIONAMIENTO"]) | df["Condición"].isna()
    if "Estado" in df.columns:
        mask &= df["Estado"].isin(["ACTIVO", "OPERATIVO", "EN FUNCIONAMIENTO"]) | df["Estado"].isna()
    if "Institución" in df.columns:
        mask &= df["Institución"].str.contains("GOBIERNO|MINSA", case=False, na=False) | df["Institución"].isna()

    df = df[mask].copy()
    print(f"[Hosp] Tras filtrar por operativo/institución: {len(df)}")

    # construir GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[lon_col], df[lat_col]), crs=CRS_WGS84)
    gdf = gdf[gdf.geometry.is_valid].drop_duplicates(subset=["geometry"]).reset_index(drop=True)
    print(f"[Hosp] GeoDataFrame final: {len(gdf)}")
    return gdf

def cargar_centros_poblados(shp_path):
    """Carga shapefile de centros poblados, usa centroides si no son puntos, devuelve GeoDataFrame en WGS84."""
    if not shp_path.exists():
        raise FileNotFoundError(f"No se encontró el shapefile de centros poblados en {shp_path}")
    gdf = gpd.read_file(shp_path)
    print(f"[Pob] Registros originales shapefile: {len(gdf)}")
    gdf.columns = [c.strip() for c in gdf.columns]
    # asegurar CRS como WGS84
    if gdf.crs is None:
        gdf = gdf.set_crs(CRS_WGS84)
    else:
        gdf = gdf.to_crs(CRS_WGS84)

    # si no son puntos, usar centroides
    if not set(gdf.geometry.geom_type.unique()).issubset({"Point"}):
        gdf["geometry"] = gdf.geometry.centroid
        print("[Pob] Convertidos a centroides (no eran puntos)")

    gdf = gdf[gdf.geometry.is_valid].drop_duplicates(subset=["geometry"]).reset_index(drop=True)
    print(f"[Pob] GeoDataFrame final (sin duplicados, válidos): {len(gdf)}")
    return gdf

def proximidad_por_region(pop_gdf, hosp_gdf, region_name, buffer_m=10000):
    """Cuenta hospitales dentro de buffer_m por cada centro poblado en region_name."""
    # detectar columna que indica departamento/provincia
    possible_region_cols = [c for c in pop_gdf.columns if c.upper() in ["DEP", "DEPARTAMENTO", "DEPARTAMEN", "DEPT", "DEPTOS", "DEPTO", "DEPART", "DEPARTAMEN"]]
    if not possible_region_cols:
        # intentar columnas con 'DEP' en el nombre
        possibles = [c for c in pop_gdf.columns if "DEP" in c.upper()]
        possible_region_cols = possibles
    if not possible_region_cols:
        raise ValueError("No se encontró columna de departamento en pop_gdf. Columnas disponibles: " + ", ".join(pop_gdf.columns))

    region_col = possible_region_cols[0]
    print(f"[Prox] Usando columna de región: {region_col}")

    pop_sel = pop_gdf[pop_gdf[region_col].str.contains(region_name, case=False, na=False)].copy()
    print(f"[Prox] Centros seleccionados en {region_name}: {len(pop_sel)}")
    if pop_sel.empty:
        return {"pop_sel": pop_sel, "min": None, "max": None}

    # proyectar a métrico para buffers
    pop_sel = pop_sel.to_crs(CRS_UTM18S)
    hosp_p = hosp_gdf.to_crs(CRS_UTM18S)
    sidx = hosp_p.sindex

    counts, buffers, hosp_ids_list = [], [], []
    for idx, row in pop_sel.iterrows():
        buf = row.geometry.buffer(buffer_m)
        buffers.append(buf)
        possible_idx = list(sidx.intersection(buf.bounds))
        possible = hosp_p.iloc[possible_idx]
        inside = possible[possible.geometry.within(buf)]
        counts.append(len(inside))
        hosp_ids_list.append(list(inside.index))

    pop_sel["hosp_in_10km"] = counts
    pop_sel["buffer_geom"] = gpd.GeoSeries(buffers, crs=CRS_UTM18S).to_crs(CRS_WGS84).values
    pop_sel["hosp_ids"] = hosp_ids_list
    pop_sel = pop_sel.to_crs(CRS_WGS84)

    # min/max (manejar None si todos tienen NaN etc)
    if pop_sel["hosp_in_10km"].size == 0:
        return {"pop_sel": pop_sel, "min": None, "max": None}
    min_row = pop_sel.loc[pop_sel["hosp_in_10km"].idxmin()]
    max_row = pop_sel.loc[pop_sel["hosp_in_10km"].idxmax()]

    return {"pop_sel": pop_sel, "min": min_row, "max": max_row}

def construir_mapa_folium(center_row, hosp_gdf, out_html, circle_color="red"):
    """Crea mapa folium con centro, buffer y hospitales dentro."""
    if center_row is None:
        print("[Map] center_row es None, no se crea mapa.")
        return None
    m = folium.Map(location=[center_row.geometry.y, center_row.geometry.x], zoom_start=11)
    # añadir buffer
    folium.GeoJson(center_row["buffer_geom"].__geo_interface__,
                   style_function=lambda feat: {"color": circle_color, "fill": False, "weight": 2}).add_to(m)
    # marcador del centro
    folium.Marker([center_row.geometry.y, center_row.geometry.x],
                  popup=f'{center_row.get("NOM_POBLAD", "N/A")} - {int(center_row["hosp_in_10km"])} hospitales').add_to(m)
    # hospitales
    mc = MarkerCluster()
    for hid in center_row.get("hosp_ids", []):
        if hid in hosp_gdf.index:
            h = hosp_gdf.loc[hid]
            mc.add_child(folium.Marker([h.geometry.y, h.geometry.x],
                                       popup=h.get("Nombre del establecimiento", "Hospital")))
    m.add_child(mc)
    m.save(out_html)
    print(f"[Map] Guardado mapa: {out_html}")
    return m

# 3) Ejecución principal (Lima y Loreto)

def ejecutar_tareas_proximidad():
    # verificar inputs
    if not HOSP_CSV.exists():
        raise FileNotFoundError(f"No existe CSV de hospitales: {HOSP_CSV}")
    if not POP_FILE.exists():
        raise FileNotFoundError(f"No existe shapefile de centros poblados: {POP_FILE}")

    print("[Run] Cargando hospitales...")
    hospitals_gdf = cargar_hospitales_csv(HOSP_CSV)

    print("[Run] Cargando centros poblados...")
    pop_gdf = cargar_centros_poblados(POP_FILE)

    # limpieza final (ya se hace dentro de loaders, pero repito por seguridad)
    hospitals_gdf = hospitals_gdf.drop_duplicates(subset=["geometry"]).reset_index(drop=True)
    pop_gdf = pop_gdf[pop_gdf.geometry.is_valid].reset_index(drop=True)

    # imprimir CRS para evidencia
    print("[Run] CRS hospitales:", hospitals_gdf.crs)
    print("[Run] CRS centros poblados:", pop_gdf.crs)

    regions = ["LIMA", "LORETO"]
    outputs_dict = {}

    for region in regions:
        print(f"\n[Run] Procesando región: {region} ...")
        res = proximidad_por_region(pop_gdf, hospitals_gdf, region, buffer_m=10000)
        pop_sel, min_row, max_row = res["pop_sel"], res["min"], res["max"]

        if pop_sel.empty:
            print(f"[Run] No hay centros poblados para {region} (o filtro no los encontró). Saltando.")
            continue

        # guardar resumen CSV
        out_csv = OUT_DIR / f"proximity_summary_{region}.csv"
        pop_sel[["NOM_POBLAD", "hosp_in_10km"]].to_csv(out_csv, index=False, encoding="utf-8")
        print(f"[Run] Guardado CSV: {out_csv.resolve()}")

        # mapas minima y maxima
        out_html_min = OUT_DIR / f"proximity_{region}_min.html"
        out_html_max = OUT_DIR / f"proximity_{region}_max.html"
        if min_row is not None:
            construir_mapa_folium(min_row, hospitals_gdf, out_html_min, "red")
        if max_row is not None:
            construir_mapa_folium(max_row, hospitals_gdf, out_html_max, "green")

        outputs_dict[region] = {"summary": out_csv.resolve(),
                                 "map_min": out_html_min.resolve() if min_row is not None else None,
                                 "map_max": out_html_max.resolve() if max_row is not None else None}
        print(f"[Run] {region} procesado. Centros analizados: {len(pop_sel)}")

    print("\n[Run] Proceso completado. Archivos escritos en:", OUT_DIR.resolve())
    # listar archivos generados para verificación
    generated = list(OUT_DIR.glob("*proximity*"))
    print("[Run] Archivos proximity generados:", [p.name for p in generated])
    return outputs_dict

outputs = ejecutar_tareas_proximidad()


Project root detectado en: c:\GitHub\Hospitals-Access-Peru
DATA_DIR: c:\GitHub\Hospitals-Access-Peru\data
SHAPE_DIR: c:\GitHub\Hospitals-Access-Peru\data\shape_file
OUT_DIR: c:\GitHub\Hospitals-Access-Peru\outputs
Archivo hospitales esperado: c:\GitHub\Hospitals-Access-Peru\data\hospitales.csv exists: True
Archivo centros poblados esperado: c:\GitHub\Hospitals-Access-Peru\data\shape_file\CCPP_IGN100K.shp exists: True
[Run] Cargando hospitales...
[Hosp] Registros originales: 20819
[Hosp] Con coordenadas válidas: 7956
[Hosp] En rango Perú: 0
[Hosp] Valores detectados (Estado/Condición/Institución): {'Estado': []}
[Hosp] Tras filtrar por operativo/institución: 0
[Hosp] GeoDataFrame final: 0
[Run] Cargando centros poblados...
[Pob] Registros originales shapefile: 136587
[Pob] GeoDataFrame final (sin duplicados, válidos): 136543
[Run] CRS hospitales: EPSG:4326
[Run] CRS centros poblados: EPSG:4326

[Run] Procesando región: LIMA ...
[Prox] Usando columna de región: DEP
[Prox] Centros selecci

## Mapeo interactivo con Folium


### Tarea 1: Coropleta Nacional (Nivel Distrital)

In [ ]:
# Verificación del UBIGEO
distritos["UBIGEO"] = distritos["UBIGEO"].astype(str).str.zfill(6)
hospitales["UBIGEO"] = hospitales["UBIGEO"].astype(str).str.zfill(6)

# Conteo de hospitales por distrito
hospital_count = hospitales.groupby("UBIGEO").size().reset_index(name="Hosp_Count")
districts = distritos.merge(hospital_count, on="UBIGEO", how="left")
districts["Hosp_Count"] = districts["Hosp_Count"].fillna(0)

import folium
from folium.plugins import MarkerCluster

# Crear mapa base
m = folium.Map(location=[-9.19, -75.0152], zoom_start=5, tiles="cartodbpositron")

# Choropleth
folium.Choropleth(
    geo_data=districts,
    data=districts,
    columns=["UBIGEO", "Hosp_Count"],
    key_on="feature.properties.UBIGEO",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color="white",
    legend_name="Número de hospitales por distrito",
).add_to(m)

# Cluster de hospitales
marker_cluster = MarkerCluster().add_to(m)
for _, row in hospitales.iterrows():
    if pd.notnull(row["NORTE"]) and pd.notnull(row["ESTE"]):
        folium.Marker(
            location=[row["NORTE"], row["ESTE"]],  # [lat, lon]
            popup=f"{row['Nombre del establecimiento']} ({row['Distrito']})"
        ).add_to(marker_cluster)

m.save("outputs/mapa_hospitales_distritos.html")

### Tarea 2: Visualización de proximidad — Lima y Loreto

In [ ]:
tooltip = "Click"

# Cargar centros poblados (con la función que definiste antes)
popcenters = cargar_centros_poblados(POP_FILE)

# Filtrar Lima y Loreto
pop_lima = popcenters[popcenters["DEP"] == "LIMA"].to_crs(32718)
pop_loreto = popcenters[popcenters["DEP"] == "LORETO"].to_crs(32718)

# Hospitales proyectados
hosp_proj = gpd.GeoDataFrame(
    hospitales,
    geometry=gpd.points_from_xy(hospitales["ESTE"], hospitales["NORTE"]),
    crs="EPSG:4326"
).to_crs(32718)

# Función para contar hospitales en buffer de 10 km
def contar_hospitales(poblado, hospitales):
    buffer = poblado.geometry.buffer(10000)  # 10 km
    return hospitales[hospitales.intersects(buffer)].shape[0]

# Calcular hospitales cercanos
pop_lima["hospitales_10km"] = pop_lima.apply(lambda row: contar_hospitales(row, hosp_proj), axis=1)
pop_loreto["hospitales_10km"] = pop_loreto.apply(lambda row: contar_hospitales(row, hosp_proj), axis=1)

# Selección de extremos
lima_min = pop_lima.loc[pop_lima["hospitales_10km"].idxmin()]
lima_max = pop_lima.loc[pop_lima["hospitales_10km"].idxmax()]
loreto_min = pop_loreto.loc[pop_loreto["hospitales_10km"].idxmin()]
loreto_max = pop_loreto.loc[pop_loreto["hospitales_10km"].idxmax()]

# Reproyectar a WGS84 para Folium
lima_min_geom = gpd.GeoSeries([lima_min.geometry], crs=32718).to_crs(4326).iloc[0]
lima_max_geom = gpd.GeoSeries([lima_max.geometry], crs=32718).to_crs(4326).iloc[0]
loreto_min_geom = gpd.GeoSeries([loreto_min.geometry], crs=32718).to_crs(4326).iloc[0]
loreto_max_geom = gpd.GeoSeries([loreto_max.geometry], crs=32718).to_crs(4326).iloc[0]

# Crear mapa combinado
m = folium.Map(location=[-9, -75], zoom_start=5, tiles="cartodbpositron")

# Lima: menos hospitales
folium.Circle(
    [lima_min_geom.y, lima_min_geom.x],
    popup=f"📍 {lima_min['NOM_POBLAD']} (Lima) - {lima_min['hospitales_10km']} hospitales",
    radius=10000, color="red", fill=True, tooltip=tooltip
).add_to(m)

# Lima: más hospitales
folium.Circle(
    [lima_max_geom.y, lima_max_geom.x],
    popup=f"📍 {lima_max['NOM_POBLAD']} (Lima) - {lima_max['hospitales_10km']} hospitales",
    radius=10000, color="green", fill=True, tooltip=tooltip
).add_to(m)

# Loreto: menos hospitales
folium.Circle(
    [loreto_min_geom.y, loreto_min_geom.x],
    popup=f"📍 {loreto_min['NOM_POBLAD']} (Loreto) - {loreto_min['hospitales_10km']} hospitales",
    radius=10000, color="red", fill=True, tooltip=tooltip
).add_to(m)

# Loreto: más hospitales
folium.Circle(
    [loreto_max_geom.y, loreto_max_geom.x],
    popup=f"📍 {loreto_max['NOM_POBLAD']} (Loreto) - {loreto_max['hospitales_10km']} hospitales",
    radius=10000, color="green", fill=True, tooltip=tooltip
).add_to(m)

m.save("outputs/proximity_combinado.html")



[Pob] Registros originales shapefile: 136587
[Pob] GeoDataFrame final (sin duplicados, válidos): 136543
